# Q1. Embedding the query

In [2]:
from fastembed import TextEmbedding
import numpy as np

In [3]:
model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en', cache_dir='.cache')
query = 'I just discovered the course. Can I join now?'

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

In [4]:
embeddings = list(model.embed([query]))
q = embeddings[0]

In [5]:
print(q.shape)
print(np.min(q))

(512,)
-0.11726373885183883


# Q2. Cosine similarity with another vector

In [6]:
doc = 'Can I still join the course after the start date?'
doc_vec = list(model.embed(doc))[0]

In [7]:
similarity = np.dot(q, doc_vec)
print(similarity)

0.9008528895674548


# Q3. Ranking by cosine

In [8]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [9]:
text_embed = list(model.embed([doc['text'] for doc in documents]))

sims = np.dot(text_embed, q)
print(np.argmax(sims))

1


# Q4. Ranking by cosine, version two

In [10]:
full_embed = list(model.embed([doc['question'] + ' ' + doc['text'] for doc in documents]))

sims_full = np.dot(full_embed, q)
print(np.argmax(sims_full))

0


In `Q3`, we computed the cosine similarity between the query and the document embeddings using only the `'text'` field. The most relevant document was found at index 1, which clearly aligns in content and context with the query. In `Q4`, we modified the input by concatenating the `'question'` and `'text'` fields before embedding. Interestingly, this change caused the highest similarity to shift to index 0, which is less relevant to the query. This shows that adding more context doesn't always improve vector search results. In some cases, concatenating multiple fields may dilute the semantic meaning or shift the focus of the embedding, resulting in less accurate similarity scores.

# Q5. Selecting the embedding model

In [11]:
model_small = TextEmbedding(model_name='BAAI/bge-small-en') 
sample = list(model_small.embed(['test']))[0]
print(len(sample))

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

384


# Q6. Indexing with Qdrant

In [12]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [13]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [14]:
client = QdrantClient(host="localhost", port=6333)

client.recreate_collection(
    collection_name="zoomcamp_faq",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

/tmp/ipykernel_11426/2227435119.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [15]:
points = []
for idx, doc in enumerate(documents):
    text = doc['question'] + " " + doc['text']
    vec = list(model_small.embed([text]))[0]
    points.append(PointStruct(id=idx, vector=vec, payload=doc))

client.upsert(collection_name="zoomcamp_faq", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
query_vec = list(model_small.embed(["I just discovered the course. Can I join now?"]))[0]
res = client.search(collection_name="zoomcamp_faq", query_vector=query_vec, limit=1)
print(res[0].score)

0.8703172


/tmp/ipykernel_11426/1814204446.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  res = client.search(collection_name="zoomcamp_faq", query_vector=query_vec, limit=1)
